# Práctica 2: Algoritmos metaheurísticos

## Sistemas Inteligentes

### Curso académico 2022-2023

#### Profesorado:

* Juan Carlos Alfaro Jiménez (`JuanCarlos.Alfaro@uclm.es`)
* Guillermo Tomás Fernández Martín (`Guillermo.Fernandez@uclm.es`)
* María Julia Flores Gallego (`Julia.Flores@uclm.es`)
* Ismael García Varea (`Ismael.Garcia@uclm.es`)
* Luis González Naharro (`Luis.GNaharro@uclm.es`)
* Aurora Macías Ojeda (`Profesor.AMacias@uclm.es`)
* Marina Sokolova Sokolova (`Marina.Sokolova@uclm.es`)

## 0. Preliminares

Antes de comenzar con el desarrollo de esta práctica es necesario **descargar**, en **formato `.py`**, el código de la **práctica anterior** con el nombre **`utils.py`**. Para ello, pulsamos, en la libreta de la primera práctica, **`File > Export .py`**.

Una vez hemos descargado y nombrado correctamente el fichero, lo **añadimos** al espacio de trabajo de la **libreta** de la práctica **actual** a través de **`Attached data > Notebook files > Upload files`** y subimos el fichero `utils.py` descargado en el paso anterior.

Tras esto, debemos **cambiar** el **constructor** de la clase **`Problem`** para que **reciba** directamente el **problema** a resolver, **en lugar de cargarlo desde** un **fichero**. Esto se debe a que los algoritmos metaheurísticos van a tener que resolver, en múltiples ocasiones, este problema. De esta manera nos **evitamos** la **carga computacional extra** que implica **leer** el problema desde un **fichero**. Además, también es necesario **comentar** cualquier línea de **código** que **imprima estadísticas** para evitar salidas largas.

## 1. Introducción

En esta práctica, vamos a **resolver** un **problema** de **optimización combinatoria** mediante **algoritmos metaheurísticos**. En particular, vamos a implementar **algoritmos genéticos** para abordar el **problema** de **ruteo** de **vehículos**. En este, **varios vehículos** con **capacidad limitada** deben **recoger paquetes** en **diferentes ubicaciones** y **trasladarlos** a una **sede central** de recogida.

Además, se **analizará** y **comparará** el **rendimiento** de **diferentes algoritmos genéticos** (mediante la modificación de los pasos correspondientes) en diferentes instancias del problema.

---

## 2. Descripción del problema

El concepto de mapa que vamos a utilizar es similar al de la primera práctica. Este se representa mediante un **grafo**, donde los **nodos** representan **ciudades** y los **enlaces** indican la existencia de una **carretera en ambos sentidos** entre dos ciudades. Además, los **enlaces** tienen un **peso** asociado indicando la **distancia real** entre las dos ciudades. Al mismo tiempo, se proporciona una **sede central**, que se trata de la ciudad donde se deben dejar los paquetes. A su vez, se dispone de una **flota de vehículos** con **capacidad limitada** que deben **recoger** los **paquetes** en las ciudades correspondientes y que **inicialmente** están aparcados en la **sede central**. **En caso de que a la hora de recoger un paquete se supere la capacidad del vehículo correspondiente, este debe volver a la sede central a descargar todos los paquetes, considerando el coste que implicaría volver**.

Un mapa es un problema en particular, pero diferentes paquetes, capacidades de vehículos y ubicación de la sede central pueden dar lugar a diferentes instancias del problema. Por tanto, el **objetivo** en este problema es **recoger todos los paquetes de tal manera que los vehículos recorran la menor distancia posible**.

**Con el objetivo de simplificar la práctica en evaluación continua, se asume que se cuenta con un único vehículo. No obstante, esto podría cambiar para la evaluación no continua.**

---

## 3. Desarrollo de la práctica

Durante el desarrollo de la práctica se va a proporcionar un conjunto de mapas, sobre los cuáles se debe resolver el problema de optimización combinatoria correspondiente. Es importante destacar que la **dimensionalidad** del **problema** (número de ciudades, carreteras y paquetes) puede ser **variable**, por lo que los diferentes **algoritmos genéticos** deben ser lo suficientemente **eficientes** para que puedan **resolver** los **problemas** en un **tiempo razonable**.

**Además, algunos escenarios se van a guardar para las entrevistas de prácticas, por lo que el código debe ser lo más general posible para cargarlos de manera rápida y sencilla.**

### 3.1. Entrada

Cada escenario tendrá un fichero `.json` asociado con la siguiente estructura:

```JSON
{
    "map": {
        "cities": [
            {
                "id": id_city_0,
                "name": name_city_0,
                "lat": latitude_city_0,
                "lon": longitude_city_0
            }
        ],
        "roads": [
            {
                "origin": origin_city_id,
                "destination": destination_city_id,
                "distance": road_distance
            }
        ]
    },
    "warehouse": warehouse_city_id,
    "vehicles": [
        {
            "id": id_vehicle_0,
            "capacity": capacity_vehicle_0
        }
    ]
    "parcels": [
        {
            "id": id_parcel_0,
            "city": parcel_city_id,
            "weight": weight_parcel_0
        }
    ]
}
```

Hay cuatro elementos principales en el fichero:

* `map`: Un diccionario con el mapa, cuya descripción es la misma que la de la primera práctica
* `warehouse`: Identificador de la ciudad donde se encuentra la sede central
* `vehicles`: Lista de vehículos disponibles
* `parcels`: Lista de paquetes a recoger

Por su parte, `vehicles` contiene:

* `id`: Identificador del vehículo
* `capacity`: Capacidad máxima del vehículo

Y `parcels`:

* `id`: Identificador del paquete
* `city`: Ciudad donde se encuentra el paquete
* `weight`: Peso del paquete

**Para añadir los ficheros con los problemas al espacio de trabajo se debe usar el mismo procedimiento anterior.**

---

## 4. Plan de trabajo

### 4.1. Formalización del problema

Para resolver cualquier problema de optimización combinatoria en primer lugar hay que definir como vamos a **codificar** las **soluciones** al problema. Si bien es algo que se deja a criterio propio, se plantea la siguiente pregunta, **¿cuál puede ser la mejor representación para una secuencia de paquetes a recoger?**

Se puede comprobar si la respuesta es correcta introduciéndola en la variable `answer` del siguiente fragmento de código:

In [36]:
# Third party
import hashlib
import json 
import random
import numpy as np

In [37]:
check_answer = lambda answer, hashed: "The answer is " + ("" if hashlib.md5(answer).hexdigest() == hashed else "in") + "correct."

In [38]:
# TODO: Introduce here the answer to use for the hashing
answer = "Hello"

# Avoid case sensitivity in the answer
answer = str.lower(answer)

# Encode the answer before hashing
answer = answer.encode("utf-8")

hashed = "90d377b31e1ac26d0d10d5612ce33ccc"  # The hashed answer
print(hashed)

check_answer(answer, hashed)

90d377b31e1ac26d0d10d5612ce33ccc


'The answer is incorrect.'

### 4.2. Implementación

A continuación se proporciona la estructura de clases recomendada para resolver el problema en cuestión. Tendréis que completar las siguientes clases de acuerdo con los algoritmos estudiados en teoría. **Debéis incluir en la siguiente celda todas las librerías que vayáis a utilizar para mantener la libreta lo más organizada posible**:

In [39]:
# Importamos la clase utils.py en la carpeta actual
import utils as ut

#### Clase `Individual`

Esta clase proporciona la **codificación** de un **individuo** de la **población**.

Los **métodos obligatorios** que se deben añadir son:

* ``__init__(self, num_genes, generation_type, crossover_type, mutation_type)``: Inicializa el **número** de **genes** del **individuo** y el **tipo** de **operación** de **generación**, **cruce** y **mutación**. Ademas, genera la **solución** que **representa** el **individuo**.
* ``generate(num_genes, generation_type)``: Método estático para **generar** una **solución** del tamaño proporcionado de acuerdo con el tipo de operación de generación.
* ``crossover(self, individual)``: **Cruza** el **individuo actual** con el **individuo** de **entrada** de acuerdo con el tipo de operación de cruce.
* ``mutation(self)``: **Muta** el **individuo** de acuerdo con el tipo de operación de mutación.
* ``evaluate(self, problem)``: **Evalua** el **individuo** usando el **problema** a **resolver**.

Y los **métodos recomendados** son:

* ``__str__(self)``: **Representación** en formato de **cadena** de **caracteres** de un **individuo**. Método útil para depurar una lista de individuos.
* ``__repr__(self)``: **Método** invocado cuando se ejecuta **``print``** sobre el **individuo**. Método útil para depurar un solo individuo.

In [40]:
class Individual:

    # =============================================================================
    # Constructor
    # =============================================================================

    # Creamos un objeto de random para poder utilizarlo en los metodos estaticos
    #random = random.Random(123)

    def __init__(self, num_genes, generation_type="random", crossover_type = None, mutation_type = None):
        self.num_genes = num_genes
        self.generation_type = generation_type
        self.crossover_type = crossover_type
        self.mutation_type = mutation_type

        self.fitness = 0
        
        self.genes = []


    # =============================================================================
    # Mandatory methods
    # =============================================================================

    # Método estático para generar una solución del tamaño proporcionado de acuerdo con el tipo de operación 
    # de generación.
    # A este metodo le pasamos num_genes, generation_type y un objeto de la clase random
    # @staticmethod
    def generate(num_genes, generation_type="random"):
        # Hay diferentes tipos de generación de individuos -> aleatoria, por distribucion, por enfoque, por seleccion...
        
        # Generación aleatoria -> Se generan los genes aleatoriamente
        
        if generation_type == "random":
            # Creamos un individuo vacio
            individual = Individual(num_genes, generation_type)

            # Generamos los genes aleatoriamente
            # Para ello utilizaremos la funcion random de la libreria random
            # for i in range(num_genes):
            #     individual.genes.append(random.random())

            # Generamos los genes aleatoriamente de tipo entero entre el rango de 0 a num_genes
            # for i in range(num_genes):
                # individual.genes.append(random.randint(0, num_genes - 1))  

            # Generamos los genes de forma aleatoria sin que se repitan
            individual.genes = random.sample(range(num_genes), num_genes)

        return individual
         
        

    # Método para realizar el cruce de dos individuos de acuerdo con el tipo de operación de cruce.
    def crossover(self, individual):
        # Tenemos diferentes tipos de operadores de cruce nosotros utilizaremos el que se utiliza para situaciones 
        # reales, el cruce 2PCX o cruce de dos puntos de cortes

        # Creamos los puntos de corte aleatoriamente, para hacer que siempre se generen dos puntos de corte debemos evitar
        # que los puntos de corte sean iguales y que los puntos de corte sean el primer o ultimo gen y que los puntos de corte
        # es decir que no sean 0 o num_genes - 1 

        cut_points = random.sample(range(1, self.num_genes - 1), 2)

        cut_points = random.sample(range(self.num_genes), 2)
        # Asignamos los puntos de corte segun el orden de menor a mayor
        cut_point_1, cut_point_2 = min(cut_points), max(cut_points)

        

        # Creamos los hijos
        child_1 = self.genes[:cut_point_1] + individual.genes[cut_point_1:cut_point_2] + self.genes[cut_point_2:]
        child_2 = individual.genes[:cut_point_1] + self.genes[cut_point_1:cut_point_2] + individual.genes[cut_point_2:]

        return child_1, child_2





    # Método para realizar la mutación de un individuo de acuerdo con el tipo de operación de mutación.
    def mutation(self):
        
        # Tenemos diferentes mecanismos de mutación para este problema utilizaremos permutaciones, en concreto
        # Intercambio e inserción
        # 1. Intercambio -> Seleccionamos dos genes aleatoriamente y los intercambiamos
        # 2. Inserción -> Seleccionamos un gen aleatoriamente y lo insertamos en una posición aleatoria
        

        # Intercambio de genes utilizando libreria random con shuffle y choice
        # Intercambio de genes

        # Seleccionamos el tipo de mutación
        mutation_type = random.randint(0, 1)

        # mutation_type = 1 

        # Intercambio
        if mutation_type == 0:
            # print("\nINTERCAMBIO\n")
            # Seleccionamos dos genes aleatoriamente
            genes = random.sample(range(self.num_genes), 2)
            # Intercambiamos los genes
            self.genes[genes[0]], self.genes[genes[1]] = self.genes[genes[1]], self.genes[genes[0]]
        
        # Inserción
        else:
            # print("\nINSERCION\n")
            # Seleccionamos un gen aleatoriamente
            gene = random.randint(0, self.num_genes - 1)
            # print("GEN: ", gene)
            # Seleccionamos una posición aleatoria
            position = random.randint(0, self.num_genes - 1)
            # print("POSICION: ", position)
            # Insertamos el gen en la posición
            self.genes.insert(position, self.genes.pop(gene))

        return self.genes

        

        
        
    
    def evaluate(self, problem):
        
        # Evaluamos el individuo
        self.fitness = problem.evaluate(self.genes)

        return self.fitness
        
        
    

    # =============================================================================
    # Recommended methods
    # =============================================================================
    

    def __str__(self):
        return f' Individual: {self.genes}'

    def __repr__(self):
        return f' Individual: {self.genes}'

    pass

**Se recomienda que se prueben cada uno de los métodos implementados de manera individual en las siguientes líneas de código:**

In [41]:
# TODO: Test here the methods to generate a solution

# Creamos un individuo
individual = Individual.generate(10, "random")

# Imprimimos el individuo
print(individual)




 Individual: [8, 0, 6, 7, 5, 4, 1, 2, 9, 3]


In [42]:
# TODO: Test here the methods to cross individuals

# Creamos dos individuos
individual_1 = Individual.generate(10, "random")
individual_2 = Individual.generate(10, "random")

# Imprimimos los individuos
print("Individuo 1\n", individual_1)
print("Individuo 2\n", individual_2)

# Realizamos el cruce
child_1, child_2 = individual_1.crossover(individual_2)

# Imprimimos los hijos
print("Hijo 1", child_1)
print("Hijo 2", child_2)


Individuo 1
  Individual: [0, 7, 5, 4, 6, 1, 3, 8, 2, 9]
Individuo 2
  Individual: [5, 0, 9, 3, 4, 8, 1, 2, 7, 6]
Hijo 1 [0, 7, 5, 4, 4, 8, 1, 2, 2, 9]
Hijo 2 [5, 0, 9, 3, 6, 1, 3, 8, 7, 6]


In [43]:
# TODO: Test here the methods to mutate an individual

# Creamos un individuo
individual = Individual.generate(5, "random")

# Imprimimos el individuo
print("Individuo\n", individual)

# Realizamos la mutación
individual.mutation()

# Imprimimos el individuo mutado
print("Individuo mutado\n", individual)



Individuo
  Individual: [0, 4, 3, 2, 1]
Individuo mutado
  Individual: [0, 3, 4, 2, 1]


#### Clase `Genetic`

Esta clase implementa un **esquema** básico de **algoritmo genético**.

Los **métodos obligatorios** que se deben añadir son:

* ``def __init__(self, population_size, num_generations, selection_type, crossover_type, crossover_probability, mutation_type, mutation_probability, keep_elitism, random_state)``: Inicializa el **tamaño** de la **población**, el **tipo** de **operación** de **selección**, **cruce**, y **mutación**, así como la **probabilidad** de aplicar las operaciones de **cruce** y **mutación**. Además, también inicializa el **número** de **mejores soluciones** de la **población actual** que se **mantienen** en la **siguiente población** y una **semilla** para garantizar que los **experimentos** son **reproducibles**. **Nótese que puede ser necesario añadir más argumentos si así se requiere**.
* ``def __call__(self, problem)``: **Método** que se **ejecuta** cuando se llama a un **objeto** de la **clase como** si fuese una **función**. En este **método** se debe **implementar** el **esquema básico** de un **algoritmo genético** que se encargue de ejecutar los pasos correspondientes. 
* ``def generate_population(self, problem)``: **Genera** la **población inicial** de acuerdo con el **problema** a resolver.
* ``def select_population(self, population, scores)``: **Selecciona** los **padres** a utilizar para la operación de cruce.
* ``def crossover(self, population)``: **Cruza pares** de **padres** teniendo en cuenta la probabilidad de cruce.
* ``def mutation(self, population)``: **Muta** los **individuos cruzados** teniendo en cuenta la probabilidad de mutación.
* ``def evaluate(self, population, problem)``: **Evalua** los **nuevos individuos** de acuerdo con el problema a resolver.
* ``def combine(self, population)``: **Forma** la **nueva generación** de acuerdo con el número de mejores individuos de la población actual a mantener en la siguiente.

In [44]:
class Genetic:

    # =============================================================================
    # Constructor
    # =============================================================================

    def __init__(self, population_size, num_generations, selection_type, crossover_type, crossover_probability, mutation_type, mutation_probability, keep_elitism, random_state):
        
        # Tamaño de la población
        self.population_size = population_size

        # Numero de generaciones
        self.num_generations = num_generations

        # Tipo de selección
        self.selection_type = selection_type

        # Tipo de cruce
        self.crossover_type = crossover_type

        # Probabilidad de cruce
        self.crossover_probability = crossover_probability

        # Tipo de mutación
        self.mutation_type = mutation_type

        # Probabilidad de mutación
        self.mutation_probability = mutation_probability

        # Si queremos mantener el elitismo, es decir, el mejor individuo de la población anterior 
        self.keep_elitism = keep_elitism
        
        # Semilla aleatoria, la semilla aleatoria es un número que se utiliza para inicializar un generador de números pseudoaleatorios.
        # Esto nos ayuda a obtener los mismos resultados cada vez que ejecutamos el algoritmo
        random.seed(random_state)



    def __call__(self, problem):
        # self.problem = problem
        # Call es el encargado de llamar a los métodos de la clase Genetic y de esta forma poder ejecutar el algoritmo genético
        # Para ello seguiremos los siguientes pasos:

        # 1. Inicializar la población: utilizar el método generate_population para generar una población de individuos de tamaño 
        # population_size, utilizando el problema especificado en el argumento problem.

        # 2. Evaluar la población: utilizar el método evaluate para evaluar a cada individuo de la población y obtener sus puntuaciones.

        # 3. Repetir el siguiente proceso durante un número determinado de generaciones (num_generations):

        # 4. Seleccionar individuos de la población: utilizar el método select_population para seleccionar un conjunto de individuos de la 
        # población utilizando el método de selección especificado en el argumento selection_type.

        # 5. Aplicar crossover: utilizar el método crossover para aplicar el tipo de crossover especificado en el argumento crossover_type a 
        # los individuos seleccionados, con una probabilidad especificada en el argumento crossover_probability.

        # 6. Aplicar mutación: utilizar el método mutation para aplicar el tipo de mutación especificado en el argumento mutation_type a los 
        # individuos resultantes del crossover, con una probabilidad especificada en el argumento mutation_probability.

        # 7. Evaluar la nueva población: utilizar el método evaluate para evaluar a cada individuo de la nueva población y obtener sus puntuaciones.

        # 8. Combinar la población: utilizar el método combine para combinar la población actual con la nueva población generada, 
        # teniendo en cuenta el argumento keep_elitism para determinar si se deben conservar los individuos más aptos de la población actual.
                

        # 1. Inicializar la población
        population = self.generate_population(problem)

        # 2. Evaluar la población
        # scores es una lista con las puntuaciones de cada individuo de la población
        scores = self.evaluate(population, problem)

        # 3. Repetir el siguiente proceso durante un número determinado de generaciones (num_generations)
        for _ in range(self.num_generations):
                
                # 4. Seleccionar individuos de la población
                selected_population = self.select_population(population, scores)
    
                # 5. Aplicar crossover
                crossed_population = self.crossover(selected_population)
    
                # 6. Aplicar mutación
                mutated_population = self.mutation(crossed_population)
    
                # 7. Evaluar la nueva población
                new_scores = self.evaluate(mutated_population, problem)
    
                # 8. Combinar la población
                population, scores = self.combine(population, scores, mutated_population, new_scores)

        # Devolvemos la mejor solución
        return population[np.argmax(scores)]





    # =============================================================================
    # Mandatory methods
    # =============================================================================

    def generate_population(self, problem):
        
        # Creamos la problacion inicial
        # population = [Individual.generate(problem.num_genes, "random") for _ in range(self.population_size)]
        population = [Individual.generate(problem, "random") for _ in range(self.population_size)]


        return population



    def select_population(self, population, scores=None, num_individuos=None):
        
        # Tenemos diferentes tipos de selección para este problema utilizaremos la selección por torneo
        # 1. Selección por torneo -> Seleccionamos dos individuos aleatoriamente y el mejor de los dos pasa a la siguiente generación
        # 3. Selección por ranking -> Se seleccionan individuos de forma proporcional a su puntuacion
        # 4. Selección por elitismo -> 

        # Ademas en la seleccion se pueden devolver una cantidad fija de individuos, un porcentaje de individuos o todos los individuos


        

        # Seleccionamos el tipo de selección
        if self.selection_type == "Torneo":
            # print("\nSELECCION POR TORNEO\n")
            # Creamos una lista para almacenar los individuos seleccionados
            selected_population = []



            # Seleccionamos cuatro individuos aleatoriamente
            selected_individuals = random.sample(population, 4)

            # Seleccionamos los dos mejores individuos de los seleccionados aleatoriamente pero de forma separada,
            # es decir, seleccionamos el mejor de los dos primeros y el mejor de los dos segundos
            selected_population.append(max(selected_individuals[:2], key=lambda x: x.fitness))
            selected_population.append(max(selected_individuals[2:], key=lambda x: x.fitness))

            return selected_population

        elif self.selection_type == "Ruleta":
            pass

        elif self.selection_type == "Ranking":
            # print("\nSELECCION POR RANKING\n")
            pass
        elif self.selection_type == "Elitismo":
            # print("\nSELECCION POR ELITISMO\n")
            pass
        


    def crossover(self, population):
        # Cruza pares de padres teniendo en cuenta la probabilidad de cruce.
        # Para la seleccion de los padres utilizamos el metodo select_population
        # Para el cruce utilizamos el metodo crossover de la clase Individual

        # El cruce se realiza teniendo en cuenta la probabilidad de cruce self.crossover_probability
        # Si la probabilidad de cruce es mayor que el numero aleatorio generado entonces se realiza el cruce
        # Si la probabilidad de cruce es menor que el numero aleatorio generado entonces no se realiza el cruce
        
        # Creamos un numero aleatorio entre 0 y 1 para realizar la probabilidad de cruce
        random_number = random.random()
        if random_number < self.crossover_probability:
            # Seleccionamos los padres
            parents = self.select_population(population)
            # Realizamos el cruce del padre 1 con el padre 2 
            child1, child2 = parents[0].crossover(parents[1])
            # Añadimos los hijos a la lista de hijos
            childs = [child1, child2]    
            

        return childs




    def mutation(self, population):
        # Añadimos una probabilidad de mutación
        for individual in population:
            if random.random() < self.mutation_probability:
                individual.mutate()

        return population

    def evaluate(self, population, problem):
        # Evaluamos los nuevos individuos de acuerdo con el problema a resolver, devolviendo una lista con las puntuaciones de cada individuo
        scores = [problem.evaluate(individual) for individual in population]

        return scores

    def combine(self, population, scores, new_population, new_scores):
        # Forma la nueva generación de acuerdo con el número de mejores individuos de la población actual a mantener en la siguiente.
        # La poblacion devuelta debe tener el mismo tamaño que la población inicial.
        
        # Lo primero que debemos hacer es ordenar la población actual y la nueva población por su puntuación (score y new_scores) de mayor a menor
        # Score es la puntuacion de la poblacion actual y new_scores es la puntuacion de la nueva poblacion, estas son asociadas por el indice del array

        # Ordenamos la población actual y la nueva población por su puntuación (score y new_scores) de mayor a menor
        

        

        # Creamos una lista para almacenar la nueva población
        new_population = []





        

        




**Se recomienda que se prueben cada uno de los métodos implementados de manera individual en las siguientes líneas de código:**

In [45]:
# TODO: Test here the methods to generate a population
# Probamos el metodo para generar una población inicial de Genetic 
# self, population_size, num_generations, selection_type, crossover_type, crossover_probability, mutation_type, mutation_probability, keep_elitism, random_state)
# A genetic le pasamos estos parametros
gn = Genetic(10, 10, 'Torneo', 'crossover', 0.8, 'mutation', 0.1, True, 1)

# Creamos una población inicial
population = gn.generate_population(10)

# Imprimimos la población inicial
print('Población inicial')
for individual in population:
    print(individual.genes)





Población inicial
[2, 1, 4, 0, 3, 5, 7, 9, 8, 6]
[1, 7, 0, 6, 3, 5, 8, 2, 9, 4]
[3, 1, 5, 0, 6, 7, 4, 8, 9, 2]
[0, 8, 3, 6, 7, 5, 1, 4, 9, 2]
[7, 4, 0, 3, 8, 9, 1, 2, 6, 5]
[5, 8, 6, 4, 9, 1, 2, 7, 3, 0]
[9, 0, 7, 1, 5, 3, 4, 2, 8, 6]
[8, 5, 1, 3, 9, 4, 0, 6, 7, 2]
[7, 0, 9, 8, 2, 4, 3, 5, 6, 1]
[8, 3, 0, 6, 1, 4, 5, 9, 2, 7]


In [46]:
# TODO: Test here the methods to select individuals
# Probamos los metodos de seleccion de individuos 
# Para probar los metodos de seleccion de individuos, vamos a crear una poblacion de 10 individuos y vamos a seleccionar 5 individuos de la poblacion de 10 individuos

# Probamos el metodo para seleccionar los padres
parents = gn.select_population(population)

# Imprimimos los padres seleccionados
print('Padres seleccionados')
for individual in parents:
    print(individual.genes) 


Padres seleccionados
[8, 5, 1, 3, 9, 4, 0, 6, 7, 2]
[2, 1, 4, 0, 3, 5, 7, 9, 8, 6]


In [ ]:
# Probramos el metodo para cruzar los padres



In [212]:
# TODO: Test here the methods to combine populations



#### Clase `CVRP`

Esta clase representa el problema en cuestión, esto es, el **problema** de **ruteo** de **vehículos**.

Los **métodos obligatorios** que se deben añadir son:

* ``def __init__(self, filename, algorithm)``: Inicializa el **problema** en cuestión y el **algoritmo** a usar para resolverlo. A su vez, se debe crear un **diccionario** que contenga como **clave** un **identificador** de **paquete** y como **valor** una **tupla** con la **ciudad** donde se encuentra dicho paquete y su **peso**.
* ``def __call__(self)``: **Resuelve** el **problema** en cuestión.
* ``def evaluate(self, solution)``: **Evalua** una **solución** para el **problema** en cuestión, **teniendo en cuenta** las **restricciones correspondientes**.
* ``def search(self, departure, goal)``: **Resuelve** un **problema** de **búsqueda** de **caminos** dada las **ciudades** de **salida** y **meta**.

**Nótese que se puede crear una estructura de datos para agilizar el proceso de búsqueda de caminos requerido por el algoritmo ¿cuál puede ser?**

In [213]:
# TODO: Introduce here the answer to use for the hashing
answer = "Hello"

# Avoid case sensitivity in the answer
answer = str.lower(answer)

# Encode the answer before hashing
answer = answer.encode("utf-8")

encoded = "0fea6a13c52b4d4725368f24b045ca84"  # The hashed answer
print(encoded)

check_answer(answer, encoded)

0fea6a13c52b4d4725368f24b045ca84


'The answer is incorrect.'

In [214]:
class CVRP:

    # =============================================================================
    # Constructor
    # =============================================================================
    
    # Sobreescribir el método __init__ de la clase problem ubicada en utils.py




# No necesitamos la primera funcion de carga de archivo, la ciudad objetivo ni ciudad inicial
    def __init__(self, filename, algorithm):

        # Inicializamos el problema y el algoritmo a usar para resolverlo
        filename = "Spain/example.json"

        with open(filename, 'r', encoding='utf8') as file:
            problem = json.load(file)
        

        self.p1 = ut.Problem(problem)
        
        # Para el problema tenemos 4 elementos principales (JSON):
        # map -> mapa del problema (Práctia 1)
        # warehouses -> almacenes del problema (En este caso solo hay 1)
        # vehicles -> lista de vehículos disponibles
            # id -> identificador del vehículo
            # capacity -> capacidad del vehículo
        # parcels -> lista de paquetes a recoger
            # id -> identificador del paquete
            # city -> ciudad donde se encuentra el paquete 
            # weight -> peso del paquete
        
        self.warehouse = problem['warehouse']
        self.vehicles = problem['vehicles']
        self.parcels = problem['parcels']
        
        # 
        self.diccionario_paquetes = {}


    # Resuelve el problema en cuestion
    def __call__(self):
        pass

    # =============================================================================
    # Mandatory methods
    # =============================================================================

    # TODO: Code here the mandatory methods

    pass

In [215]:
p1 = CVRP()
p1.vehicles[0]['capacity']


TypeError: CVRP.__init__() missing 2 required positional arguments: 'filename' and 'algorithm'

**Se recomienda que se prueben cada uno de los métodos implementados de manera individual en las siguientes líneas de código:**

In [ ]:
# TODO: Test here the method to initialize the capacited vehicle routing problem

In [ ]:
# TODO: Test here the method to solve the search problem

In [ ]:
# TODO: Test here the method to solve the capacited vehicle routing problem

### 4.3. Estudio y mejora de los algoritmos

Una vez que los algoritmos han sido implementados, se debe **estudiar** su **rendimiento**. Para ello, se debe comparar la **calidad** de las **soluciones obtenidas**, así como las **diferentes estadísticas** que se consideren adecuadas (número de generaciones, tiempo de ejecución, etc.). Factores como el tamaño máximo del problema que se soporta sin un error de memoria, así como el efecto temporal de usar escenarios más complejos son otros factores a tener en cuenta. Además, se **pueden proponer** y se valorarán la incorporación de **técnicas** que **permitan acelerar** la **ejecución** de los **algoritmos**.

---

In [ ]:
# TODO: Experiment here with the small problem

In [ ]:
# TODO: Experiment here with the medium problem

In [ ]:
# TODO: Experiment here with the large problem

### 5. Entrega y evaluación

Al igual que la práctica anterior, esta se debe **hacer en pares**. No obstante, en **casos excepcionales** se permite realizarla **individualmente**. **La fecha límite para subir la práctica es el 18 de diciembre de 2022 a las 23:55**. Las **entrevistas y evaluaciones** se realizarán la **semana siguiente**.

Algunas consideraciones:

* **En caso de que no se haya entregado la primera práctica, o se haya sacado menos de un cuatro, se podrán entregar conjuntamente en esta fecha. No obstante, se considerará únicamente un 90% de la nota global de prácticas**.
* Está práctica supone el **70%** de la **nota** en este apartado.
* La práctica se **evaluará** mediante una **entrevista individual** con el profesorado. Las fechas de las entrevistas se publicarán con antelación.
*  Se proporcionará un **conjunto** de **casos** de **prueba preliminares** (varios mapas e instancias) que se **deben resolver correctamente**. En caso contrario, la práctica se considerará suspensa.
* La **entrevista** consistirá en una serie de **preguntas** acerca del **código**.

**Por último, para la evaluación no continua se requirirá la implementación del algoritmo de búsqueda por ascenso de colinas. Además, este se deberá utilizar para inicializar la población del algoritmo genético, en lugar de que sea aleatoria.**